In [1]:
import math
from tqdm import tqdm
import numpy as np
import glob
import csv
import cv2
from PIL import Image

**Scale down** the road data to form an image of roads with a resolution of around 50m x 50m per pixel  
The resulting image is of 656x656 dimentions

In [66]:
print("Data loading and Management")
ml_road_dir = "../ml_preds_csv/"

base_file_name = "3001120103"

# 82*8 = 656
# 164*8 = 1312
grid = np.zeros((1312, 1312), dtype=np.uint8)
road_coords = dict()

# level_a = {'0': (0, 0), '1': (0, 328), '2': (328, 0), '3': (328, 328)}
level_a = {'0': (0, 0), '1': (0, 656), '2': (656, 0), '3': (656, 656)}
# level_b = {'0': (0, 0), '1': (0, 164), '2': (164, 0), '3': (164, 164)}
level_b = {'0': (0, 0), '1': (0, 328), '2': (328, 0), '3': (328, 328)}
# level_c = {'0': (0, 0), '1': (0, 82), '2': (82, 0), '3': (82, 82)}
level_c = {'0': (0, 0), '1': (0, 164), '2': (164, 0), '3': (164, 164)}
for a in tqdm(level_a):
    a_i = level_a[a][0]
    a_j = level_a[a][1]
    for b in level_b:
        b_i = a_i + level_b[b][0]
        b_j = a_j + level_b[b][1]
        for c in level_c:
            c_i = b_i + level_c[c][0]
            c_j = b_j + level_c[c][1]

            # generate files names
            file_name = base_file_name + a + b + c + ".csv"
            file_path = ml_road_dir + file_name

            with open(file_path, 'r') as csv_file:
                reader = csv.reader(csv_file)
                first_row = True
                for row in reader:
                    if first_row:
                        first_row = False
                        continue

                    d_i = int(row[0])
                    d_j = int(row[1])

                    i = c_i + int(d_i / 50)
                    j = c_j + int(d_j / 50)

                    grid[i][j] = max(np.uint8(row[2]), grid[i][j])
                    if int(row[2]) >= 75 and (i*1312+j not in road_coords or road_coords[i*1312+j][0] < int(row[2])):
                            road_coords[i*1312+j] = (int(row[2]), float(row[3]), float(row[4]))
                            



  0%|          | 0/4 [00:00<?, ?it/s]

Data loading and Management




 25%|██▌       | 1/4 [01:48<05:24, 108.31s/it]

 50%|█████     | 2/4 [04:04<03:53, 116.75s/it]

 75%|███████▌  | 3/4 [05:46<01:52, 112.32s/it]

100%|██████████| 4/4 [08:34<00:00, 128.54s/it]


In [69]:
grid[grid < 75] = 0
img = Image.fromarray(grid)
img.show()
print("Image Size:", np.shape(grid))
print("total pixels: ", 1312*1312)
print("road pixels:", np.count_nonzero(grid))

Image Size: (1312, 1312)
total pixels:  1721344
road pixels: 233706


In [70]:
roads = grid.copy()
roads[roads >= 75] = 1
print("Image Size:", np.shape(roads))
print("total pizels: ", 1312*1312)
print("road pixels:", np.count_nonzero(roads))

# visited = np.zeros(np.shape(roads))
# print("Image Size:", np.shape(visited))
# print("total pizels: ", 656*656)
# print("road pixels:", np.count_nonzero(visited))

Image Size: (1312, 1312)
total pizels:  1721344
road pixels: 233706


In [71]:
# load test cases
test_cases = []
with open("../routing_challenge_pairs.csv", 'r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        test_cases.append(row)
        
del test_cases[0]

In [72]:
# find the coordinate (i,j) of a lat/long pair in the scaled down image
def closest_ij(lati, lngi):
    min_dist = 999999
    i=0
    j=0
    lati = float(lati)*100000
    lngi = float(lngi)*100000
    for coord in road_coords:
        lat = float(road_coords[coord][1])*100000
        lng = float(road_coords[coord][2])*100000
        if min_dist > math.sqrt((lati - lat)**2 + (lngi - lng)**2):
            min_dist = math.sqrt((lati - lat)**2 + (lngi - lng)**2)
            i = int(coord/1312)
            j = int(coord - i*1312)
    return i,j

In [73]:
# An example
lat1 = test_cases[0][0]
lng1 = test_cases[0][1]
i1,j1 = closest_ij(lat1, lng1)
print("1:",(i1,j1))
print("1:", grid[i1][j1])

lat2 = test_cases[0][2]
lng2 = test_cases[0][3]
i2,j2 = closest_ij(lat2, lng2)
print("2:",(i2, j2))
print("2:", grid[i2][j2])

1: (783, 915)
1: 247
2: (1124, 836)
2: 245


Apply **Breath First Search (BFS)** to calculate the minimum number of road pixels between the two image pixels  
If no path is found, return -1

In [74]:
def BFS(i1, j1, i2, j2):
    visited = np.zeros(np.shape(roads))
    queue = []
    queue.append((0, i1, j1))
    visited[i1][j1] = 1
    front = 0
    kernel = [-1,0,1]
    while front != len(queue):
        (d, i, j) = queue[front]
        front += 1
        if i==i2 and j==j2:
            return d
        
        for p in kernel:
            for q in kernel:
                if 0<= i+p < 1312 and 0<= j+q < 1312 and roads[i+p][j+q] != 0 and visited[i+p][j+q] == 0:
                    queue.append((d+1, i+p, j+q))
                    visited[i+p][j+q] = 1
                    
    return -1

In [77]:
# test a sample
dist = BFS(783, 915, 1124, 836)
dist*25

11825

In [78]:
expected = 0
for degree in range(5, 46, 5):
    rad1 = degree * math.pi / 180
    rad2 = (degree-5) * math.pi / 180
    expected += 2*(1/math.cos(rad1))*(math.sin(rad1) - math.sin(rad2))
expected

1.6028889820468648

In [ ]:
output_file = open("../results/results_part2_50", 'w')
csv_writer = csv.writer(output_file)
csv_writer.writerow(["latitude_src", "longitude_src", "latitude_dst", "longitude_dst", "distance"])
num_faults = 0
for case in tqdm(test_cases):
    lat1 = case[0]
    lng1 = case[1]
    i1,j1 = closest_ij(lat1, lng1)

    lat2 = case[2]
    lng2 = case[3]
    i2,j2 = closest_ij(lat2, lng2)
    
    dist = BFS(i1, j1, i2, j2)
    
    if dist != -1:
        dist = dist*25
    else:
        num_faults += 1
        
    csv_writer.writerow([lat1, lng1, lat2, lng2, str(dist)])
output_file.close()
print("Number of faults:", num_faults)




  0%|          | 0/1000 [00:00<?, ?it/s]


  0%|          | 1/1000 [00:02<33:41,  2.02s/it]


  0%|          | 2/1000 [00:03<30:29,  1.83s/it]


  0%|          | 3/1000 [00:03<24:04,  1.45s/it]


  0%|          | 4/1000 [00:05<23:25,  1.41s/it]


  0%|          | 5/1000 [00:06<22:49,  1.38s/it]


  1%|          | 6/1000 [00:10<33:04,  2.00s/it]


  1%|          | 7/1000 [00:10<24:22,  1.47s/it]


  1%|          | 8/1000 [00:12<29:43,  1.80s/it]


  1%|          | 9/1000 [00:15<32:06,  1.94s/it]


  1%|          | 10/1000 [00:17<36:01,  2.18s/it]


  1%|          | 11/1000 [00:18<26:29,  1.61s/it]


  1%|          | 12/1000 [00:19<25:03,  1.52s/it]


  1%|▏         | 13/1000 [00:22<30:23,  1.85s/it]


  1%|▏         | 14/1000 [00:24<31:13,  1.90s/it]


  2%|▏         | 15/1000 [00:25<27:38,  1.68s/it]


  2%|▏         | 16/1000 [00:26<23:43,  1.45s/it]


  2%|▏         | 17/1000 [00:28<26:04,  1.59s/it]


  2%|▏         | 18/1000 [00:30<32:15,  1.97s/it]


  2%|▏         | 19/1000 [0

In [59]:
math.pi

3.141592653589793

In [65]:
1312/2

656.0